En este experimento se desea probar la búsqueda semántica en un espacio reducido pero mayor a un único vector, como se probó en el experimento 2. 

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import os
import sys
import redis
from datetime import datetime
from dotenv import load_dotenv
sys.path.append('/Users/alexander/Projects/hyperpersonalization/app/tools')
from vectordb import connect_to_redis, check_index_existance, add_message_to_db

load_dotenv()

redis_host = os.getenv("REDIS_HOST") or "localhost"
redis_port = os.getenv("REDIS_PORT") or 6379
index_name = "conversations"
index_path = "../indexes/conversations"
embeddings = OpenAIEmbeddings()

db = FAISS.load_local(folder_path=index_path, index_name=index_name, embeddings=embeddings)


Ingresamos información en la base semántica como extractos de conversaciones.


A cada fragmento se le calculan los embeddings y se guardan junto con el texto original en la base. Siempre asociadas al identificador de la persona y el nombre gracias al uso de las metadatas.

Esto permite compartimentalizar la información y filtrarla por usuario.

In [6]:
metadatas = [{"person_id": "alexander.ditzend", "person_name": "Alexander"}]
timestamp = datetime.now().isoformat()
message = {"timestamp": timestamp, "content": "mi madre se llama Alicia, tiene 66, es jubilada" }
text = str(message)
db.add_texts(texts=[text], embedding=embeddings, metadatas=metadatas)

['2b4e7776-2ba1-40c6-a5b8-4c52c7bfd09d']

In [9]:
metadatas = [{"person_id": "alexander.ditzend", "person_name": "Alexander"}]
timestamp = datetime.now().isoformat()
message = {"timestamp": timestamp, "content": "mis hijas se llaman Aurora y Alba, Aurora tiene 12 y Alba 9" }
text = str(message)
db.add_texts(texts=[text], embedding=embeddings, metadatas=metadatas)

['8365053e-e582-4bdb-9892-36d2499a78a1']

In [12]:
metadatas = [{"person_id": "alexander.ditzend", "person_name": "Alexander"}]
timestamp = datetime.now().isoformat()
message = {"timestamp": timestamp, "content": "a mis hijas no les gusta estar en el pasillo, siempre ventana" }
text = str(message)
db.add_texts(texts=[text], embedding=embeddings, metadatas=metadatas)

['89b188cb-5eed-44fe-92d8-cc62c077e676']

Ahora comenzamos a hacer preguntas dentro de "query" y observamos la respuesta de la base. En orden de similitud recibimos los vectores que podemos usar para componer una respuesta o usar en algún proceso de razonamiento con un LLM.

In [10]:
query = "cuantos años tiene la madre del cliente?"
filter = {"person_id": "alexander.ditzend"}
db.similarity_search_with_score(query=query, embeddings=embeddings, filter=filter, k=1)

[(Document(page_content="{'timestamp': '2023-07-12T12:20:08.065627', 'content': 'mi madre se llama Alicia, tiene 66, es jubilada'}", metadata={'person_id': 'alexander.ditzend', 'person_name': 'Alexander'}),
  0.43335056)]

In [14]:
query = "tiene hijos?"
filter = {"person_id": "alexander.ditzend"}
db.similarity_search_with_score(query=query, embeddings=embeddings, filter=filter, k=2)

[(Document(page_content="{'timestamp': '2023-07-12T12:23:34.853229', 'content': 'a mis hijas no les gusta estar en el pasillo, siempre ventana'}", metadata={'person_id': 'alexander.ditzend', 'person_name': 'Alexander'}),
  0.45402932),
 (Document(page_content="{'timestamp': '2023-07-12T12:22:19.835459', 'content': 'mis hijas se llaman Aurora y Alba, Aurora tiene 12 y Alba 9'}", metadata={'person_id': 'alexander.ditzend', 'person_name': 'Alexander'}),
  0.46825188)]

In [15]:
query = "cuales son las preferencias de los hijos?"
filter = {"person_id": "alexander.ditzend"}
db.similarity_search_with_score(query=query, embeddings=embeddings, filter=filter, k=2)

[(Document(page_content="{'timestamp': '2023-07-12T12:23:34.853229', 'content': 'a mis hijas no les gusta estar en el pasillo, siempre ventana'}", metadata={'person_id': 'alexander.ditzend', 'person_name': 'Alexander'}),
  0.48871845),
 (Document(page_content="{'timestamp': '2023-07-12T12:22:19.835459', 'content': 'mis hijas se llaman Aurora y Alba, Aurora tiene 12 y Alba 9'}", metadata={'person_id': 'alexander.ditzend', 'person_name': 'Alexander'}),
  0.53449)]

In [16]:
query = "edad de la hija mayor"
filter = {"person_id": "alexander.ditzend"}
db.similarity_search_with_score(query=query, embeddings=embeddings, filter=filter, k=2)

[(Document(page_content="{'timestamp': '2023-07-12T12:22:19.835459', 'content': 'mis hijas se llaman Aurora y Alba, Aurora tiene 12 y Alba 9'}", metadata={'person_id': 'alexander.ditzend', 'person_name': 'Alexander'}),
  0.42045504),
 (Document(page_content="{'timestamp': '2023-07-12T12:23:34.853229', 'content': 'a mis hijas no les gusta estar en el pasillo, siempre ventana'}", metadata={'person_id': 'alexander.ditzend', 'person_name': 'Alexander'}),
  0.44310012)]

En todos los casos las respuestas de la base son correctas, se ha comprobado en este experimento que la búsqueda por similitud usando embeddings puede ser aplicable al objetivo que se busca en este trabajo. 

Sigue la misma consideración que en los experimentos anteriores, esto es, que las conclusiones obtenidas de un universo tan diminuto no pueden extrapolarse a un escenario real, órdenes de magnitud superior en cantidad de vectores. 